In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import math
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread, imsave, figure
import pandas as pd
import numpy as np
import os
import sys
from skimage.color import rgb2gray
from skimage.transform import rescale, resize


#Keras:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

#Custom Functions:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

from imports.utils.log_progress import log_progress
from imports.utils.visualization import Visualize, Evaluate
from imports.utils.enums import DATA_BASE_PATH, SHAPE
from imports.models.u_net import get_unet
from imports.utils.utils import rle_encode, rle_decode

DATA_IMAGE_PATH = DATA_BASE_PATH + '/Images'
DATA_MASK_PATH = DATA_BASE_PATH + '/Masks'

Using TensorFlow backend.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
train_df = pd.read_pickle(DATA_BASE_PATH+'/train_df')
val_df = pd.read_pickle(DATA_BASE_PATH+'/val_df')
test_df = pd.read_pickle(DATA_BASE_PATH+'/test_df')

print("Number of training Samples:", len(train_df))
print("Number of validation Samples:", len(val_df))
print("Number of test Samples:", len(test_df))

Number of training Samples: 354
Number of validation Samples: 102
Number of test Samples: 51


#### Models:

| Model | Description | Filename |
|:------|------|--|
|   UNet_Mask  | Unet- Model trained on Handlabeled Mask-Data 'train_df'| unet1024_60epochs.h5 |
|   UNet_Circle  | Unet- Model trained on Handlabeled Mask-Data 'train_df'| - |

In [6]:
from imports.models.losses import bce_dice_loss, dice_coeff, iou
from keras.models import load_model, model_from_json

import keras.losses
keras.losses.custom_loss = bce_dice_loss

model = load_model('../saved_models/unet/rgb_512_786_seperate_outputs/unet_dice_0.8199_iou_0.9352.h5', custom_objects={'bce_dice_loss': bce_dice_loss,'dice_coeff':dice_coeff,'iou':iou})

### 1. Unet_Mask:

In [48]:
ev_params = {'df':train_df,
             'input_shape':(512,768,3),
             'pred_layer' : 1,
             'model':model}
ev = Evaluate(**ev_params)

#### 1.1 Dice Coeff:

In [18]:
unet_mask_dice_min, unet_mask_dice_max, unet_mask_dice_avg = ev.get_dice_coeff(mode='simple')

Average prediction time: 0.40 s


In [22]:
unet_mask_dice_avg

0.77673237193956612

In [ ]:
imgs = np.array[]

In [49]:
_ = ev.mean_average_precicion()

Thresh	TP	FP	FN	Prec.
0.500	0	2	4	0.000
0.550	0	2	4	0.000
0.600	0	2	4	0.000
0.650	0	2	4	0.000
0.700	0	2	4	0.000
0.750	0	2	4	0.000
0.800	0	2	4	0.000
0.850	0	2	4	0.000
0.900	0	2	4	0.000
0.950	0	2	4	0.000
AP	-	-	-	0.000
0.0
Thresh	TP	FP	FN	Prec.
0.500	4	7	3	0.286
0.550	3	8	4	0.200
0.600	2	9	5	0.125
0.650	2	9	5	0.125
0.700	1	10	6	0.059
0.750	0	11	7	0.000
0.800	0	11	7	0.000
0.850	0	11	7	0.000
0.900	0	11	7	0.000
0.950	0	11	7	0.000
AP	-	-	-	0.079
0.125
Thresh	TP	FP	FN	Prec.
0.500	10	17	11	0.263
0.550	9	18	12	0.231
0.600	5	22	16	0.116
0.650	5	22	16	0.116
0.700	5	22	16	0.116
0.750	4	23	17	0.091
0.800	1	26	20	0.021
0.850	0	27	21	0.000
0.900	0	27	21	0.000
0.950	0	27	21	0.000
AP	-	-	-	0.095
0.116279069767
Thresh	TP	FP	FN	Prec.
0.500	7	3	1	0.636
0.550	7	3	1	0.636
0.600	7	3	1	0.636
0.650	7	3	1	0.636
0.700	7	3	1	0.636
0.750	5	5	3	0.385
0.800	5	5	3	0.385
0.850	3	7	5	0.200
0.900	2	8	6	0.125
0.950	0	10	8	0.000
AP	-	-	-	0.428
0.636363636364
Thresh	TP	FP	FN	Prec.
0.500	3	3	2	0.375
0.550	3	3	2	0.375
0.600	

KeyboardInterrupt: 

### 2. Unet_Circle:

In [ ]:
ev_params = {'df':test_df,
             'model':unet_circle}
eval_unet_mask = Evaluate(**ev_params)

In [76]:
unet_circle_dice_min, unet_circle_dice_max, unet_circle_dice_avg = ev.get_dice_coeff(mode='simple')

0.90322095098287847

### Results:

| Model | Dice-Coefficient | Root- Position Accuracy |
|:------|------|--|
|   UNet_Mask  | {{ unet_mask_dice_avg }} | unet1024_60epochs.h5 |
|   UNet_Circle  | Unet- Model trained on Handlabeled Mask-Data 'train_df'| - |

In [30]:
class User:
    def __init__(self, name):
        self.name = name
        self.age = None
    
    def set_age(self,age):
        self.age = age
        
    def print_age(self):
        print("Age of " + self.name + ": " + str(self.age))
 
    def printName(self):
        print("Name  = " + self.name)

In [31]:
class Programmer(User):
    def __init__(self, name):
        self.name = name
 
    def doPython(self):
        print("Programming Python")

In [33]:
sepp = User('sepp')
toni = Programmer('toni')

sepp.set_age(1)
sepp.print_age()

toni.set_age(10)
toni.print_age()

Age of sepp: 1
Age of toni: 10


In [61]:
tmp = None
for i, r in test_df.iterrows():
    tmp = r.iloc[0]
    break
print(tmp)

../data/00_train_all/images/
